In [1]:
import numpy as np
import pandas as pd
import json 

In [5]:
# import utility functions 
from ret import read_data # utility func to load data
from ret import get_id_from_info # utility func to return id by entering song's info
from ret import display_res # utility func to display results 
from ret import get_genre #utility func to get the genres from a list of id´s
from ret import get_genre_from_query #utility func to get the id and genre from the query
from ret import get_genre_from_ids #utility func to get the id and genre from the retrieved results
from ret import generate_cos_sim_matrix

# import the retrieval systems 
from ret import random_baseline # baseline retrieval system that returns random results 
from ret import text_based # modularized text based retrieval system
from ret import audio_based # modularized audio based retrieval system
from ret import video_based # modularized video based retrieval system

# import wrapper function to calculate cosine similarity
from ret import cos_sim 

# import evaluation functions 
from ret import gen_cov_10
from ret import ndcg_score
from ret import gen_div_10
from ret import calculate_precision_at_k
#from ret import calculate_recall_at_k
from ret import plot_precision_recall_curve
from ret import get_avg_recall_at_k
from ret import get_avg_precision_at_k

ImportError: cannot import name 'generate_cos_sim_matrix' from 'ret' (C:\Users\Sara\Documents\GitHub\MMSR\project_2_team_E\ret.py)

In [2]:
def calc_avg_precision_recall_at_k(df, embedding, genres, k):
    # Generate the cosine similarity matrix
    cos_sim_matrix = generate_cos_sim_matrix(embedding)

    # Define an empty list to store the precision and recall scores
    precision_scores = []
    recall_scores = []

    # Loop over each query track
    for idx, query_id in enumerate(df.id):
        # Get the row index of the query track in the cosine similarity matrix
        query_index = cos_sim_matrix.index.get_loc(query_id)

        # Get the top-k most similar tracks using the cosine similarity matrix
        similar_tracks_indices = np.argsort(cos_sim_matrix.iloc[query_index])[::-1][1:k+1]

        # Retrieve the genres of the top-k similar tracks
        retrieved_genres = [get_genre(cos_sim_matrix.index[i], genres) for i in similar_tracks_indices]

        # Get the genre of the query track
        query_genre = get_genre(query_id, genres)

        # Calculate precision and recall
        relevant_count = get_relevancy_count(query_genre, retrieved_genres)
        precision = relevant_count / k
        recall = relevant_count / len(query_genre)

        # Append the precision and recall scores to the list
        precision_scores.append(precision)
        recall_scores.append(recall)

        # Print progress
        print(f'Processed row {idx+1}/{len(df)}')

    # Calculate the average precision and recall over all query tracks
    avg_precision = np.mean(precision_scores)
    avg_recall = np.mean(recall_scores)

    # Print the results
    print(f'Average Precision@{k}: {avg_precision:.9f}')
    print(f'Average Recall@{k}: {avg_recall:.9f}')

    return avg_precision, avg_recall